In [2]:
import Base.*
import Base.convert
import Base.promote_rule

In [3]:
function printInheritance(x)
    if(x == Any)
        print(x)
    else
        printInheritance(supertype(x))
        print("-->")
        print(x)
    end
end

printInheritance(Int128)


Any-->Number-->Real-->Integer-->Signed-->Int128

In [4]:
function NWD(a, b)
    while b != 0
        c = a % b
        a = b
        b = c
    end
return a
end

NWD (generic function with 1 method)

In [5]:
struct Gn{N} 
   x :: Int64
   Gn(x) =  if (x >= N)
                throw(DomainError)
            elseif(NWD(x, N) == 1)
                 new(x)
            else throw(DomainError)
end
end

In [6]:
*(a::Gn{N}, b::Gn{N}) where {N} = Gn{N}(((a.x % N) * (b.x % N)) % N)
*(a::Gn{N}, b::T) where {N, T<:Integer} = a * Gn{N}(b)
*(a::T, b::Gn{N}) where {N, T<:Integer} = Gn{N}(a) * b

* (generic function with 185 methods)

In [7]:
convert(::Type{Gn{N}}, x::Int64) where {N} = Gn{N}(x)
convert(::Type{Int64}, x::Gn{N}) where {N} = x.x

convert (generic function with 723 methods)

In [8]:
convert(Gn{7}, 6)

Gn{7}(6)

In [9]:
convert(Int64, Gn{5}(4))

4

In [10]:
promote_rule(::Type{Gn{N}}, ::Type{T}) where{N, T<:Integer} = Int64

promote_rule (generic function with 124 methods)

In [11]:
promote_type(Gn{7}, Int64)

Int64

In [12]:
promote_type(Int8, Gn{11})

Int64

In [13]:
promote(Gn{111}(7), 8)

(7, 8)

In [14]:
function modPow(a::Gn{N}, x::T) where{N, T<:Integer}
    c = 1
    e = 0
    while x > e
        e = e + 1
        c = (c * a.x) % N
    end
    return Gn{N}(c)
end

modPow (generic function with 1 method)

In [15]:
modPow(Gn{497}(4), 13)

Gn{497}(445)

In [16]:
function ord(a::Gn{N}) where {N}
    r = 1
    pow = modPow(a, r).x
    
    while pow != 1
        r = r+1
        pow = modPow(a, r).x
    end
    return r
end


ord (generic function with 1 method)

In [17]:
ord(Gn{7}(4))

3

In [18]:
function inverse(a::Gn{N}) where {N}
    t = 0
    r = N   
    newT = 1
    newR = a.x
    while newR != 0
        quotient = div(r, newR) 
        (t, newT) = (newT, t - quotient * newT)
        (r, newR) = (newR, r - quotient * newR)
    end
    if t < 0
        t = t + N
    end
    return Gn{N}(t)
end

inverse (generic function with 1 method)

In [19]:
inverse(Gn{17}(6))

Gn{17}(3)

In [20]:
function elems(::Type{Gn{N}}) where {N}
    a = 1
    num = 0
    while a < N
        num += 1
        try
            Gn{N}(a)
        catch
            num -= 1
        end
        a += 1
    end
    return num
end

elems (generic function with 1 method)

In [21]:
elems(Gn{9})

6

In [22]:
elems(Gn{17})

16

In [25]:
N = 55
c = 17

b = 4

r = ord(Gn{N}(b))
d = inverse(Gn{N}(c)) #private key
a = modPow(Gn{N}(b), d.x)
println(a)

modPow(a, c).x

Gn{55}(9)


4